In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import time

# custom functions
from functions import *

In [2]:
df = pd.read_csv('../data/clean/dataset.csv').fillna('')

In [3]:
df

,name,url,title,lemmes,funny
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue...",new york mairie vouloir ve avenue pieton,0
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...,responsabilite social environnemental musee,0
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...,presidentielle americain frontiere mexicain es...,0
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...,mal-logement lutte habitant revendication ecol...,0
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique",warum joe punk canal historique,0
...,...,...,...,...,...
102080,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...,desormais chance gagner apres retrait biden de...,0
102081,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ...",jo paris handball tournoi terminer lille paris,0
102082,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ...",jo paris handball pays titres palmares tournoi...,0
102083,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet...",jo paris handball palmares complet equipe fran...,0


In [ ]:
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf
import numpy as np
from tqdm import tqdm

# Load the CamemBERT model and tokenizer
model_name = "camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

# Example list of 100,000 sentences (replace with your actual data)
sentences = list(df['title'])

# Define batch size for processing
batch_size = 1024  # Adjust based on available memory

# Function to extract embeddings for a batch of sentences
def extract_embeddings(batch_sentences):
    # Tokenize and encode the batch of sentences
    inputs = tokenizer(
        batch_sentences, return_tensors="tf", padding=True, truncation=True
    )
    
    # Get model outputs
    outputs = model(**inputs)
    
    # Pool the embeddings by averaging token embeddings
    sentence_embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1)
    return sentence_embeddings

# Loop through sentences in batches and store the embeddings
all_embeddings = []
num_batches = len(sentences) // batch_size + (len(sentences) % batch_size != 0)

for i in tqdm(range(0, len(sentences), batch_size), total=num_batches, desc="Encoding Batches"):
    batch_sentences = sentences[i:i + batch_size]
    batch_embeddings = extract_embeddings(batch_sentences)
    all_embeddings.append(batch_embeddings)

# Concatenate all batch embeddings into a single tensor
all_embeddings = tf.concat(all_embeddings, axis=0)

# Convert to NumPy array if needed for downstream processing
all_embeddings_np = all_embeddings.numpy()

print("Shape of embeddings:", all_embeddings_np.shape)  # Should be (100000, hidden_size)

In [ ]:
pca_graph(embeddings)

In [ ]:
y = df.funny

In [ ]:
mapper = umap.UMAP(n_components=2, n_jobs=-1, verbose=1).fit(embeddings)    

In [ ]:
umap.plot.points(mapper, labels=y, background='black', color_key_cmap='spring', width=1600, height=1600)